In [81]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

s = pd.read_csv('../data/nyc-temps.txt').squeeze()
df = DataFrame({'temp': s, 
                'hour': [0,3,6,9,12,15,18,21] * 91})

df.loc[(df['hour'] == 3) | (df['hour'] == 6), 'temp'] = np.nan
#3시와 6시 온도를 결측치로 변경.
#df.loc를 사용해 df['hour']의 값이 3 또는 6인 행의 'temp' 열을 선택하고, 해당 위치에 np.nan을 할당.


# Beyond 1

By default, the `interpolate` method tries to average the remaining values before and after any `NaN`. However, we can change how it works, by passing `method='nearest'`. Does that change our data substantially?

기본적으로 interpolate 메서드는 NaN 앞뒤의 남은 값들을 평균내어 결측치를 채웁니다. 그러나 `method='nearest'`를 전달하면 보간 방식이 변경됩니다. 이것이 우리의 데이터에 실질적인 변화를 주나요?


method (보간 방식)에는 아래와 같이 존재  
'linear' (기본값): 앞뒤 값을 직선으로 연결해서 채움  
'time': 인덱스가 datetime이면 시간 간격에 맞게 보간  
'nearest': 가장 가까운 실제 값으로 채움  
'polynomial': 다항식 보간 (order 지정 필요)  
'spline': 스플라인 보간  
'pad' / 'ffill': 결측을 앞 값으로 채움  
'backfill' / 'bfill': 결측을 뒤 값으로 채움  

In [82]:
# No, doesn't seem to change things significantly -- maybe because temperatures
# don't really vary all that much across readings.

df.interpolate(method='nearest').describe()
#nearest 방법을 사용하여 결측치를 보간하고, 결과의 기술 통계량을 계산.

#아래의 결과는 원본 데이터의 describe() 결과임
# count    728.000000
# mean      -1.050824
# std        5.026357
# min      -14.000000
# 25%       -4.000000
# 50%        0.000000
# 75%        2.000000
# max       12.000000

#nearest 방법을 사용한 보간 결과의 describe() 결과는 원본과 동일함

,temp,hour
count,728.000000,728.000000
mean,-1.050824,10.500000
std,5.026357,6.878589
min,-14.000000,0.000000
25%,-4.000000,5.250000
50%,0.000000,10.500000
75%,2.000000,15.750000
max,12.000000,21.000000


# Beyond 2

Let's assume that the equipment works fine around the clock, but that it fails to record a reading at -1 degree and below. Are the interpolated values similar to the real (missing) values they replace? Why or why not?

장비가 24시간 정상 작동한다고 가정하되, -1도 이하에서는 측정값을 기록하지 못한다고 합시다. 보간(interpolate)으로 채워진 값들은 실제(결측된) 값들과 유사한가요? 그렇다면/아니라면 그 이유는 무엇인가요?

In [83]:
# reset our data
df = DataFrame({'temp': s, 
                'hour': [0,3,6,9,12,15,18,21] * 91})

#데이터프레임을 다시 생성해 원본 상태로 되돌림

In [84]:
# Remove values <= -1 to NaN
df.loc[df['temp'] <= -1, 'temp'] = np.nan
# 온도가 -1 이하인 값을 결측치로 변경


In [85]:
# Interpolate!
df = df.interpolate()
#선형보간 수행

In [86]:
# Wow, the mean is now 2 and the median is now 1 -- significantly higher
# Not surprising, of course, given that we removed all very low temperatures!
df['temp'].describe()
#7개의 데이터가 결측치로 변경된 후 선형 보간이 수행됨.

#원본 describe()
# count    728.000000
# mean      -1.050824
# std        5.026357
# min      -14.000000
# 25%       -4.000000
# 50%        0.000000
# 75%        2.000000
# max       12.000000

#결측치 처리 후 describe() 결과는 평균 2.02, 중앙값 1로 바뀐것을 볼 수 있음.
#놀랄게 아니라 당연히 낮은 온도들이 제거되었기 때문.

count    721.000000
mean       2.022191
std        2.345483
min        0.000000
25%        0.209524
50%        1.000000
75%        3.000000
max       12.000000
Name: temp, dtype: float64

# Beyond 3

A cheap solution to interpolation is to replace `NaN` values with the column's mean. Do this, and compare the new mean and median. Again, why are (or aren't) these values similar to the original ones?


보간(interpolation)의 간단한 해결책으로는 `NaN` 값을 해당 열의 평균으로 대체하는 방법이 있습니다. 이를 수행한 다음 새 평균과 중앙값을 비교하세요. 다시 말하지만, 이 값들이 원래 값들과 유사한지(혹은 그렇지 않은지) 그리고 그 이유는 무엇인지 설명하세요.


In [87]:
# reset our data
df = DataFrame({'temp': s, 
                'hour': [0,3,6,9,12,15,18,21] * 91})

# Remove values <= -1 to NaN
df.loc[df['temp'] <= -1, 'temp'] = np.nan

df = df.fillna(df.mean())
#결측치를 평균값으로 대체

In [88]:
# Wow, these values are even worse than the interpolated ones!

# Clearly, running .interpolate is a better option than using the mean -- 
# in no small part because it calculated a local mean, rather than
# a global one across all of the data.
# 와, 이 값들은 보간된 값들보다도 오히려 더 좋지 않습니다!
# 분명히 `.interpolate`를 실행하는 것이 평균(mean)으로 대체하는 것보다 더 나은 선택입니다 
# — 그 주요 이유 중 하나는 `.interpolate`가 전체 데이터에 걸친 전역 평균이 아니라 주변(국소) 값들을 바탕으로 평균을 계산하기 때문입니다.

df.describe()
#결측치를 평균값으로 대체한 후의 describe() 결과


#원본 describe()
# count    728.000000
# mean      -1.050824
# std        5.026357
# min      -14.000000
# 25%       -4.000000
# 50%        0.000000
# 75%        2.000000
# max       12.000000


#보간법이 평균 대체법보다 더 나은 이유는 결측치 주변의 값들을 고려하여 구하는데,
#이 데이터의 경우 뉴욕의 3달치 시간별 온도이므로, 주변 값들의 데이터와 유사한 값을 가지기에
#3달 전체의 평균을 대체하는 것보다 더 적절한 값을 가지게 됨.

,temp,hour
count,728.000000,728.000000
mean,2.763926,10.500000
std,1.935689,6.878589
min,0.000000,0.000000
25%,2.000000,5.250000
50%,2.763926,10.500000
75%,2.763926,15.750000
max,12.000000,21.000000
